In [1]:
import pandas as pd

In [2]:
nacional = pd.read_csv('bne_balance_nacional.csv')
regional = pd.read_excel('bne_balance_regional.xlsx')

In [3]:
# Diccionario de coincidencias de nombres invertido para transformar regional_total
coincidencias = {
    'coque metalurgico': 'coque mineral',
    'gas de alto horno': 'gas de altos hornos',
    'gas licuado de petroleo': 'gas licuado',
    'derivados industriales de petroleo': 'd.i. de petroleo',
    'energia hidroelectrica': 'energia hidrica',
    'gas de coque': 'gas coque'
}
sectores = {
    'Comercial, público y residencial': 'Sector Cmrcl., Púb. Y Residencial',
     'Energía': 'Energético: Consumo Propio',
     'Industria y Minería': 'Industrial y Minero',
     'Transformación': 'Centros de transfornación',
     }

In [4]:
# Función para transformar los nombres en regional_total según el diccionario de coincidencias
def transformar_nombres(row,x):
    combustible = row['combustible']
    if combustible in x:
        return x[combustible]
    return combustible



In [21]:
import unicodedata
class Comparar_Balances():
  #Cargar las variables al modelo
    def __init__(self, nacional, regional,coincidencias):
        self.nacional = nacional
        self.regional = regional
        self.coincidencias = coincidencias

    def transformar_nombres(self, row):
        combustible = row['combustible']
        if combustible in self.coincidencias:
            return self.coincidencias[combustible]
        return combustible
    
    def transformar_sector(self, row):
        sector = row['Sector']
        if sector in self.sectores:
            return self.sectores[sector]
        return sector


    def comparar_balance(self):
        nacional_total = self.nacional.query('item == "CONSUMO FINAL" and categoria.str.startswith("Sector")')[['anio', 'combustible', 'tcal']]
        nacional_total.columns = ['año','combustible','tcal_nacional']
        nacional_total = nacional_total.groupby(['año','combustible']).sum().reset_index()

        regional_2 = self.regional.query('actividad != "Transformación"')[['anio','energetico','tcal']]
        regional_total = regional_2.groupby(['anio','energetico']).sum().reset_index()
        regional_total.columns = ['año','combustible','tcal_regional']

        # Transformar a minúsculas
        nacional_total['combustible_clean'] = nacional_total['combustible'].str.lower()

        # Quitar tildes
        nacional_total['combustible_clean'] = nacional_total['combustible_clean'].apply(lambda x: unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode('utf-8'))

            # Transformar a minúsculas
        regional_total['combustible_clean'] = regional_total['combustible'].str.lower()

        # Quitar tildes
        regional_total['combustible_clean'] = regional_total['combustible_clean'].apply(lambda x: unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode('utf-8'))
        
        nacional_total.drop(columns='combustible',inplace=True)
        regional_total.drop(columns='combustible',inplace=True)
        nacional_total = nacional_total[['año','combustible_clean','tcal_nacional']]
        regional_total = regional_total[['año','combustible_clean','tcal_regional']]

        nacional_total.columns = ['año','combustible','tcal_nacional']
        regional_total.columns = ['año','combustible','tcal_regional']
        
        # Aplicar la función a la columna 'combustible' del DataFrame regional_total
        regional_total['combustible'] = regional_total.apply(self.transformar_nombres, axis=1)

        df_final = pd.merge(nacional_total,regional_total,how='inner',on=['año','combustible'])
        df_final.fillna(0,inplace=True)

        df_final2 = df_final[['año','tcal_nacional','tcal_regional']].groupby('año').sum().reset_index()

        df_final['diferencia tcal'] = df_final['tcal_nacional'] - df_final['tcal_regional']
        df_final2['diferencia tcal'] = df_final2['tcal_nacional'] - df_final2['tcal_regional']

        return df_final, df_final2
    

In [23]:
balance = Comparar_Balances(nacional=nacional,regional=regional,coincidencias=coincidencias)
df_comb, df_total = balance.comparar_balance()

In [24]:
df_comb

,año,combustible,tcal_nacional,tcal_regional,diferencia tcal
0,2014,alquitran,155.099000,155.099000,0.000000e+00
1,2014,biogas,19.752756,19.752756,0.000000e+00
2,2014,biomasa,40175.348260,40175.348263,-2.900604e-06
3,2014,carbon,2353.299390,2353.299390,-4.547474e-13
4,2014,coque mineral,75.662426,75.662426,0.000000e+00
...,...,...,...,...,...
157,2021,kerosene,1606.899033,1606.899033,3.554640e-07
158,2021,kerosene de aviacion,9183.604686,9183.604686,-1.682820e-07
159,2021,nafta,99.486744,449.061824,-3.495751e+02
160,2021,petroleo combustible,6678.854834,6678.854834,3.559999e-07


In [25]:
df_total

,año,tcal_nacional,tcal_regional,diferencia tcal
0,2014,272092.958716,274382.187057,-2289.228340
1,2015,275160.288641,272331.103660,2829.184981
2,2016,280241.216082,282300.555705,-2059.339624
3,2017,287224.059165,288901.295666,-1677.236501
4,2018,298734.538452,300825.415841,-2090.877389
5,2019,299574.725421,301597.217537,-2022.492116
6,2020,281469.273988,283235.413395,-1766.139407
7,2021,291460.966634,293142.624797,-1681.658163


In [27]:
df_comb.to_csv('Balance Nacional vs Regional combustibles.csv',index=False,encoding='utf-8-sig')

In [28]:
df_total.to_csv('Balance Nacional vs Regional total.csv',index=False,encoding='utf-8-sig')

In [13]:
comb_nac = nacional_total.combustible.unique()
len(comb_nac)

NameError: name 'nacional_total' is not defined

In [130]:
comb_reg = regional_total.combustible.unique()
len(comb_reg)

28

In [131]:
# Utilizando la operación de intersección con el operador "&"
coincidencias = set(comb_nac) & set(comb_reg)

# Otra opción es utilizando el método intersection de set
# coincidencias = set(lista1).intersection(lista2)

# El resultado será una lista con los valores en común
print(coincidencias)  # Salida: {3, 4, 5}

# Para saber cuántos valores en común hay, puedes usar la función len
cantidad_coincidencias = len(coincidencias)
print(cantidad_coincidencias)  # Salida: 3

{'gasolina de motor', 'biomasa', 'petroleo diesel', 'gas natural', 'energia solar', 'energia hidrica', 'gas coque', 'nafta', 'carbon', 'coque de petroleo', 'alquitran', 'biogas', 'gas de refineria', 'gasolina de aviacion', 'petroleo combustible', 'geotermia', 'energia eolica', 'kerosene', 'coque mineral', 'gas corriente', 'gas de altos hornos', 'gas licuado', 'petroleo crudo', 'd.i. de petroleo', 'kerosene de aviacion', 'electricidad'}
26


In [132]:
# Utilizando la operación de diferencia con el operador "-"
diferencias_lista1 = set(comb_nac) - set(comb_reg)
diferencias_lista2 = set(comb_reg) - set(comb_nac)

# Otra opción es utilizando el método difference de set
# diferencias_lista1 = set(lista1).difference(lista2)
# diferencias_lista2 = set(lista2).difference(lista1)

# Los resultados serán listas con los elementos que no coinciden en cada una
print(diferencias_lista1)  # Salida: {1, 2}
print()
print(diferencias_lista2)  # Salida: {6, 7}

{'metanol'}

{'licor negro', 'petcoke'}
